### Read SLHA, SModelS output and store the data in a pandas DataFrame

In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import pandas as pd
import glob,os
import numpy as np
from pandas import json_normalize
from importlib import util
import pyslha
import sys
sys.path.append(os.path.expanduser('~/smodels'))
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
from smodels.particlesLoader import getParticlesFromSLHA

pd.options.mode.chained_assignment = None #Disable copy warnings

In [2]:
BSMlist = getParticlesFromSLHA('./2mdm_example.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
print(BSMlist)

[gha, gha~, ghz, ghz~, ghwp, ghwp~, ghwm, ghwm~, ghg, ghg~, zp, ghzp, ghzp~, sd, chi]


In [3]:
slhaFolder = '../data/slha_files'
smodelsFolder = '../data/smodels_results'
parametersSmodels = './smodels_parameters_2mdm.ini'

In [35]:
#Convert Experimental Results list to a dictionary
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades","OutputStatus",
                 "Total xsec for missing topologies (fb)", "Total xsec for missing topologies with displaced decays (fb)",
                 "Total xsec for missing topologies with prompt decays (fb)","Total xsec for topologies outside the grid (fb)"]
for f in glob.glob(smodelsFolder+'/*.py'):

    spec = util.spec_from_file_location("smodelsOutput", f)
    smodelsOutput = util.module_from_spec(spec)
    spec.loader.exec_module(smodelsOutput)
    smodelsDict = smodelsOutput.smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
        slhaFile = f.replace('.py','')
    smodelsDict['filename'] = os.path.basename(slhaFile)
    if 'CombinedRes' in smodelsDict:
        smodelsDict['ExptRes'] += smodelsDict.pop('CombinedRes')
    if 'ExptRes' in smodelsDict:
        expList = sorted(smodelsDict['ExptRes'], 
                            key=lambda pt: (pt['r'] is not None,pt['r']),
                            reverse=True)
        for iexp,exp in enumerate(expList):
            newDict = {k : v for k,v in smodelsDict.items()}
            newDict['ExptRes'] = iexp
            newDict.update({k : v for k,v in exp.items()})
            data.append(newDict)
    else:
        data.append(smodelsDict)
    if len(data) > 30:
        break

In [36]:
smDF = pd.DataFrame(data)

In [37]:
smDF.set_index(['filename','ExptRes'],inplace=True)
smDF

maxcond  theory prediction (fb)  \
filename             ExptRes                                    
scan_3_g7btudrp.slha 0            0.0               59.663240   
                     1            0.0                0.000007   
                     2            0.0                0.000013   
                     3            0.0                0.000001   
scan_1_nw8r40ie.slha 0            0.0              152.561500   
                     1            NaN                     NaN   
                     2            0.0                0.794803   
                     3            0.0              147.331100   
                     4            0.0                8.037258   
                     5            0.0               24.754500   
                     6            0.0               24.754500   
                     7            0.0                0.137623   
scan_3_zhbq06_k.slha 0            0.0              173.768300   
                     1            0.0               34.734250   
                     2            0.0               25.005440   
                     3            0.0                0.057448   
                     4            0.0                0.000088   
                     5            0.0                0.000057   
                     6            0.0                0.000006   
scan_3_ej7m2n85.slha 0            0.0               28.574400   
                     1            0.0                0.000003   
                     2            0.0                0.000007   
                     3            0.0                0.000001   
scan_3_r53swaf4.slha 0            0.0                2.873855   
                     1            0.0                0.000883   
                     2            0.0                0.001215   
                     3            0.0                0.000400   
scan_3_vk5hr1af.slha 0            0.0                9.823235   
                     1            0.0                1.964501   
                     2            0.0                9.823235   
                     3            0.0                1.964647   
                     4            0.0                0.487391   
                     5            0.0                0.000007   
                     6            0.0                0.000005   
                     7            0.0                0.000001   

                              upper limit (fb)  expected upper limit (fb)  \
filename             ExptRes                                                
scan_3_g7btudrp.slha 0               42.060800                        NaN   
                     1                0.386700                   0.331300   
                     2                0.776900                   0.792700   
                     3                0.109200                   0.097530   
scan_1_nw8r40ie.slha 0               83.050200                  95.748600   
                     1                     NaN                        NaN   
                     2                2.711473                   2.214494   
                     3              573.929000                        NaN   
                     4               33.686470                  17.669110   
                     5              123.613000                 171.890000   
                     6              139.275000                        NaN   
                     7                0.776900                   0.792700   
scan_3_zhbq06_k.slha 0               49.409700                  76.774800   
                     1               41.963000                  28.363000   
                     2              226.083000                 116.358000   
                     3              459.816000                        NaN   
                     4                1.028000                   0.894600   
                     5                0.776900                   0.792700   
                     6                0.080180                   0.127600   
scan_3_

In [16]:
print(len(data))

11


In [6]:
#Convert data to flat DataFrame:
smodelsDF = json_normalize(data)

In [7]:
#Get SLHA data:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 52])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['NPINPUTS'].items()])
    
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 52])
    BRsDict = {}
    for pdg,val in slha.decays.items():
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 1e-7: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)

In [8]:
len(slhaData)

3

In [9]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [10]:
#Merge with SModelS DataFrame
dataDF = slhaDF.merge(smodelsDF,how='inner')

In [11]:
print('Final number of data points:',dataDF.shape[0])
#print(dataDF2.columns.values.tolist()) #Print all columns names

Final number of data points: 3


In [12]:
dataDF

,filename,mass.9000006,mass.9900026,mass.9900032,mass.9000002,mass.9000003,mass.9000004,mass.9000005,width.9000002,width.9000003,...,ExptRes.result2.TxNames weights (fb).TRV1qq,ExptRes.result4.TxNames weights (fb).TRV1,ExptRes.result4.nll,ExptRes.result4.nll_min,ExptRes.result4.nll_SM,ExptRes.result5.TxNames weights (fb).TRV1,ExptRes.result5.nll,ExptRes.result5.nll_min,ExptRes.result5.nll_SM,ExptRes.result6.TxNames weights (fb).TRS1
0,scan_3_g7btudrp.slha,65.0000,1238.2430,2476.485,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,scan_1_nw8r40ie.slha,293.7867,800.1534,1600.307,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,scan_3_zhbq06_k.slha,65.0000,878.2694,1756.539,91.1876,79.82436,79.82436,1500.0,2.4952,2.085,...,25.005443,0.000088,9.413419,9.227586,9.413548,0.000057,9.200832,9.200812,9.200812,1.056680e-10
